In [ ]:
import pandas as pd
import torch
import time
import json
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [ ]:
path_raw_data = '/content/drive/MyDrive/Licenta1/Licenta/products_table.csv'
df_raw = pd.read_csv(path_raw_data)

print(f"Numărul de produse: {len(df_raw)}")
print(df_raw.head())

Numărul de produse: 62646
                                         nume_produs  id_brand nume_brand
0  Sapca Unisex Baseball BMW M Motorsport Neagra ...    7375.0        BMW
1  Prosop Audi, Albastru inchis si deschis, bumba...   21197.0       Audi
2  Tricou Personalizat AudiLove go hard, ADLER, n...   11465.0      ADLER
3          Tricou VR46 MONZA RALLY REPLICA Negru 2XL  105708.0      VR|46
4              Tricou VR46 THE DOCTOR COMIC Galben L  105708.0      VR|46


In [ ]:

model_id = "Qwen/Qwen2.5-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
few_shot_examples = [
    {"nume_produs": "Sapca Unisex Baseball BMW M Motorsport Neagra Marime M/L", "brand": "BMW"},
    {"nume_produs": "Prosop Audi, Albastru inchis si deschis, bumbac, 80 x 150 cm", "brand": "Audi"},
    {"nume_produs": "Tricou Personalizat AudiLove go hard, ADLER, negru, XL", "brand": "ADLER"},
    {"nume_produs": "Tricou VR46 MONZA RALLY REPLICA Negru 2XL", "brand": "VR|46"},
    {"nume_produs": "Breloc din aluminiu cu protectie pentru cartela masinii compatibil cu Tesla model 3/Y, negru", "brand": "OEM"},
    {"nume_produs": "Breloc Volkswagen Tiguan Metalic, Piele, Argintiu YLKRS AutoMotive", "brand": "YLKRS AUTOMOTIVE"},
    {"nume_produs": "Breloc / husa protectie cheie AUDI, cu fermoar, 8.5cm L / 5 cm l, Maro Ubitec ®", "brand": "Ubitec"},
]

def build_prompt_qwen(nume_produs, branduri_batch, examples):
    example_str = "\n".join([f"'{ex['nume_produs']}' → Brand: {ex['brand']}" for ex in examples])
    branduri_str = ", ".join(branduri_batch)

    return f"""
Your task is to extract the correct brand from a product name.

RULES:
- Choose only from the approved list.
- If multiple brands are mentioned, the correct brand is NEVER in the middle.
- Output ONLY the JSON object, like:  {{"brand": "BRAND_NAME_FROM_LIST"}}
- Replace "BRAND_NAME_FROM_LIST" with the correct brand from the approved list.

FEW-SHOT EXAMPLES:
{example_str}

PRODUCT NAME:
'{nume_produs}'

APPROVED BRANDS:
{branduri_str}
"""



In [ ]:
# Few-shot examples
few_shot_examples = [
    {"nume_produs": "Sapca Unisex Baseball BMW M Motorsport Neagra Marime M/L", "brand": "BMW"},
    {"nume_produs": "Prosop Audi, Albastru inchis si deschis, bumbac, 80 x 150 cm", "brand": "Audi"},
    {"nume_produs": "Tricou Personalizat AudiLove go hard, ADLER, negru, XL", "brand": "ADLER"},
    {"nume_produs": "Tricou VR46 MONZA RALLY REPLICA Negru 2XL", "brand": "VR|46"},
    {"nume_produs": "Breloc din aluminiu cu protectie pentru cartela masinii compatibil cu Tesla model 3/Y, negru", "brand": "OEM"},
    {"nume_produs": "Breloc Volkswagen Tiguan Metalic, Piele, Argintiu YLKRS AutoMotive", "brand": "YLKRS AUTOMOTIVE"},
    {"nume_produs": "Breloc / husa protectie cheie AUDI, cu fermoar, 8.5cm L / 5 cm l, Maro Ubitec ®", "brand": "Ubitec"},
]

def build_prompt_qwen_structured(nume_produs, branduri_batch, examples):
    example_str = "\n".join([f"'{ex['nume_produs']}' → Brand: {ex['brand']}" for ex in examples])
    branduri_str = ", ".join(branduri_batch)
    schema = json.dumps({"brand": "string"}, indent=2)

    prompt = f"""
INSTRUCTIONS:
You are an assistant that extracts brands in JSON format.

RULES:
- Select ONLY from the list below.
- If multiple brands are mentioned, the correct brand is NEVER in the middle.
- Return only the brand name, exactly as it appears in the list.
- Your output must strictly follow this JSON schema:
{schema}

FEW-SHOT EXAMPLES:
{example_str}

PRODUCT NAME:
'{nume_produs}'

APPROVED BRANDS:
{branduri_str}

Return ONLY the JSON object. Do NOT include explanations.
"""
    return prompt


In [ ]:
import json
import re
import pandas as pd
import time
import torch
import torch._dynamo


torch._dynamo.config.cache_size_limit = 64
torch._dynamo.config.suppress_errors = True


# Functie pentru parsarea JSON

def parse_json_from_response(response_text):
    matches = re.findall(r"\{.*?\}", response_text)
    for match in reversed(matches):
        try:
            data = json.loads(match)
            if "brand" in data:
                return data.get("brand", "")
        except json.JSONDecodeError:
            continue
    return ""


# Parametri generali

results = []
batch_size = 100
nr_produse = min(len(df_raw), 1000)
output_path = "/content/drive/MyDrive/Licenta1/Licenta/Rezultate_qwen_f.csv"
retry_wait_time = 1
max_retries = 5


for start in range(0, nr_produse, batch_size):
    end = start + batch_size
    df_batch = df_raw.iloc[start:end].copy()
    branduri_batch = df_batch["nume_brand"].dropna().astype(str).unique().tolist()

    print(f" Rulez batch {start} → {end-1} (cu {len(branduri_batch)} branduri)")

    for index, row in df_batch.iterrows():
        nume_produs = row['nume_produs']
        brand_corect = row['nume_brand']
        brand_extras = ""
        numar_erori = 0
        timp_executie = ""

        prompt = build_prompt_qwen(nume_produs, branduri_batch, few_shot_examples)

        for attempt in range(max_retries):
            try:
                start_t = time.time()
                output = pipe(prompt, max_new_tokens=150, do_sample=False)[0]["generated_text"]
                durata = time.time() - start_t

                print(f"\n RAW OUTPUT pentru index {index}:\n{output}\n")

                brand_extras = parse_json_from_response(output)

                if brand_extras:
                    timp_executie = f"{durata:.3f}"  # salveaza timpul doar la succes
                    break

            except Exception as e:
                print(f" Eroare la index {index}, încercarea {attempt+1}: {e}")
                brand_extras = ""

            numar_erori += 1
            if attempt < max_retries - 1:
                time.sleep(retry_wait_time)

        brand_extras_status = (
            "Corect" if brand_extras.strip().lower() == brand_corect.strip().lower() else "Incorect"
        ) if brand_extras else "Incorect"


        results.append({
            "index": index,
            "model": "Qwen2.5-7B-Instruct",
            "nume_produs": nume_produs,
            "brand_corect": brand_corect,
            "brand_extras": brand_extras,
            "brand_extras_status": brand_extras_status,
            "timp_executie_secunde": timp_executie,
            "numar_erori": numar_erori
        })


    df_partial = pd.DataFrame(results)
    df_partial.to_csv(output_path, index=False)
    print(f" Salvare intermediara: {end} produse procesate in:\n{output_path}")


print(f"\n Procesare completa! {len(results)} produse salvate in:\n{output_path}")




The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


 Rulez batch 0 → 99 (cu 24 branduri)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



 RAW OUTPUT pentru index 0:

Your task is to extract the correct brand from a product name.

RULES:
- Choose only from the approved list.
- If multiple brands are mentioned, the correct brand is NEVER in the middle.
- Output ONLY the JSON object, like:  {"brand": "BRAND_NAME_FROM_LIST"}
- Replace "BRAND_NAME_FROM_LIST" with the correct brand from the approved list.

FEW-SHOT EXAMPLES:
'Sapca Unisex Baseball BMW M Motorsport Neagra Marime M/L' → Brand: BMW
'Prosop Audi, Albastru inchis si deschis, bumbac, 80 x 150 cm' → Brand: Audi
'Tricou Personalizat AudiLove go hard, ADLER, negru, XL' → Brand: ADLER
'Tricou VR46 MONZA RALLY REPLICA Negru 2XL' → Brand: VR|46
'Breloc din aluminiu cu protectie pentru cartela masinii compatibil cu Tesla model 3/Y, negru' → Brand: OEM
'Breloc Volkswagen Tiguan Metalic, Piele, Argintiu YLKRS AutoMotive' → Brand: YLKRS AUTOMOTIVE
'Breloc / husa protectie cheie AUDI, cu fermoar, 8.5cm L / 5 cm l, Maro Ubitec ®' → Brand: Ubitec

PRODUCT NAME:
'Sapca Unisex B

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...


 RAW OUTPUT pentru index 996:

Your task is to extract the correct brand from a product name.

RULES:
- Choose only from the approved list.
- If multiple brands are mentioned, the correct brand is NEVER in the middle.
- Output ONLY the JSON object, like:  {"brand": "BRAND_NAME_FROM_LIST"}
- Replace "BRAND_NAME_FROM_LIST" with the correct brand from the approved list.

FEW-SHOT EXAMPLES:
'Sapca Unisex Baseball BMW M Motorsport Neagra Marime M/L' → Brand: BMW
'Prosop Audi, Albastru inchis si deschis, bumbac, 80 x 150 cm' → Brand: Audi
'Tricou Personalizat AudiLove go hard, ADLER, negru, XL' → Brand: ADLER
'Tricou VR46 MONZA RALLY REPLICA Negru 2XL' → Brand: VR|46
'Breloc din aluminiu cu protectie pentru cartela masinii compatibil cu Tesla model 3/Y, negru' → Brand: OEM
'Breloc Volkswagen Tiguan Metalic, Piele, Argintiu YLKRS AutoMotive' → Brand: YLKRS AUTOMOTIVE
'Breloc / husa protectie cheie AUDI, cu fermoar, 8.5cm L / 5 cm l, Maro Ubitec ®' → Brand: Ubitec

PRODUCT NAME:
'Spuma activa

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



 RAW OUTPUT pentru index 997:

Your task is to extract the correct brand from a product name.

RULES:
- Choose only from the approved list.
- If multiple brands are mentioned, the correct brand is NEVER in the middle.
- Output ONLY the JSON object, like:  {"brand": "BRAND_NAME_FROM_LIST"}
- Replace "BRAND_NAME_FROM_LIST" with the correct brand from the approved list.

FEW-SHOT EXAMPLES:
'Sapca Unisex Baseball BMW M Motorsport Neagra Marime M/L' → Brand: BMW
'Prosop Audi, Albastru inchis si deschis, bumbac, 80 x 150 cm' → Brand: Audi
'Tricou Personalizat AudiLove go hard, ADLER, negru, XL' → Brand: ADLER
'Tricou VR46 MONZA RALLY REPLICA Negru 2XL' → Brand: VR|46
'Breloc din aluminiu cu protectie pentru cartela masinii compatibil cu Tesla model 3/Y, negru' → Brand: OEM
'Breloc Volkswagen Tiguan Metalic, Piele, Argintiu YLKRS AutoMotive' → Brand: YLKRS AUTOMOTIVE
'Breloc / husa protectie cheie AUDI, cu fermoar, 8.5cm L / 5 cm l, Maro Ubitec ®' → Brand: Ubitec

PRODUCT NAME:
'Spuma activa

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



 RAW OUTPUT pentru index 998:

Your task is to extract the correct brand from a product name.

RULES:
- Choose only from the approved list.
- If multiple brands are mentioned, the correct brand is NEVER in the middle.
- Output ONLY the JSON object, like:  {"brand": "BRAND_NAME_FROM_LIST"}
- Replace "BRAND_NAME_FROM_LIST" with the correct brand from the approved list.

FEW-SHOT EXAMPLES:
'Sapca Unisex Baseball BMW M Motorsport Neagra Marime M/L' → Brand: BMW
'Prosop Audi, Albastru inchis si deschis, bumbac, 80 x 150 cm' → Brand: Audi
'Tricou Personalizat AudiLove go hard, ADLER, negru, XL' → Brand: ADLER
'Tricou VR46 MONZA RALLY REPLICA Negru 2XL' → Brand: VR|46
'Breloc din aluminiu cu protectie pentru cartela masinii compatibil cu Tesla model 3/Y, negru' → Brand: OEM
'Breloc Volkswagen Tiguan Metalic, Piele, Argintiu YLKRS AutoMotive' → Brand: YLKRS AUTOMOTIVE
'Breloc / husa protectie cheie AUDI, cu fermoar, 8.5cm L / 5 cm l, Maro Ubitec ®' → Brand: Ubitec

PRODUCT NAME:
'Spuma activa